- Welcome to Quantitative and High Frequency Trading Training Program


In [1]:
from helper import *
from stats import *
from imp import reload   
import helper
import stats
reload(helper)
reload(stats)
par_get_all_signal

<function helper.par_get_all_signal(signal_name, file_list, product, period, HEAD_PATH='d:/intern', SAVE_PATH='e:/intern', DATA_PATH='d:/intern')>

- number of cores, set this number according to your computer

In [2]:
CORE_NUM = int(os.environ['NUMBER_OF_PROCESSORS'])
CORE_NUM

16

- sample product, you can change to your specific product symbol

In [3]:
product_list = ["btc.usd.td", "eth.usd.td", "btc.usdt", "eth.usdt"]

In [4]:
import dask
from dask import compute, delayed

- sample period

In [5]:
period = 4096

In [6]:
all_dates = os.listdir(DATA_PATH + product_list[0])
all_dates.sort()
all_dates = np.array(all_dates)
n_days = len(all_dates)
n_days

1059

- choose an example day

In [7]:
train_sample = all_dates<"2020"

test_sample = all_dates>"2020"


In [8]:
date_str = [n[0:10] for n in all_dates]
format_dates = np.array([pd.to_datetime(d) for d in date_str])
SAVE_PATH = "e:/intern"

- at last we plot the selected pnl

- now we can see that the number of signals are not very big
- and the quality is not so good
- we can propose a systematic method to generate a large number of signals then
- we can choose good ones from them

- for example, we can first generate some signals about volatility without direction
- there are some ideas about it

- standard deviation
- range of price
- ratio of volume to turnover

- then we can generate some signals with direction
- actually all of our previous signals have direction
- but some of them are good and some of them are bad
- we can focus on some good and simple ones, such as

- normalized return
- rsi
- dbook

- let's try to generate range signals first

In [22]:
## calculate rolling standard deviation
from collections import OrderedDict
class foctor_std_period(factor_template):
    factor_name = "std.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(10,13)))
    ])
    
    def formula(self, data, period):
        return np.sqrt(fast_roll_var(data["wpr"], period))

In [23]:
## calcualte rolling variance
def fast_roll_var(x, period):
      x_ma = cum(x,period)/period ## roling first moment
      x2 = x*x
      x2_ma = cum(x2,period)/period ## rolling second moment
      var_x = x2_ma-x_ma*x_ma ## rolling variance
      return(var_x)

In [24]:
x20 = foctor_std_period()
for product in product_list:
    create_signal_path(x20, product, SAVE_PATH);

e:/intern/tmp pkl/btc.usd.td/std.1024
e:/intern/tmp pkl/btc.usd.td/std.2048
e:/intern/tmp pkl/btc.usd.td/std.4096
e:/intern/tmp pkl/eth.usd.td/std.1024
e:/intern/tmp pkl/eth.usd.td/std.2048
e:/intern/tmp pkl/eth.usd.td/std.4096
e:/intern/tmp pkl/btc.usdt/std.1024
e:/intern/tmp pkl/btc.usdt/std.2048
e:/intern/tmp pkl/btc.usdt/std.4096
e:/intern/tmp pkl/eth.usdt/std.1024
e:/intern/tmp pkl/eth.usdt/std.2048
e:/intern/tmp pkl/eth.usdt/std.4096


In [25]:
## rolling price range
from collections import OrderedDict
class foctor_range_period(factor_template):
    factor_name = "range.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(10,13)))
    ])
    
    def formula(self, data, period):
        return data["max."+str(period)]-data["min."+str(period)]

In [26]:
x21 = foctor_range_period()
for product in product_list:
    create_signal_path(x21, product, SAVE_PATH);

e:/intern/tmp pkl/btc.usd.td/range.1024
e:/intern/tmp pkl/btc.usd.td/range.2048
e:/intern/tmp pkl/btc.usd.td/range.4096
e:/intern/tmp pkl/eth.usd.td/range.1024
e:/intern/tmp pkl/eth.usd.td/range.2048
e:/intern/tmp pkl/eth.usd.td/range.4096
e:/intern/tmp pkl/btc.usdt/range.1024
e:/intern/tmp pkl/btc.usdt/range.2048
e:/intern/tmp pkl/btc.usdt/range.4096
e:/intern/tmp pkl/eth.usdt/range.1024
e:/intern/tmp pkl/eth.usdt/range.2048
e:/intern/tmp pkl/eth.usdt/range.4096


In [31]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, os.listdir(DATA_PATH + product)))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x21, product=product, HEAD_PATH=SAVE_PATH, n=14)

Wall time: 24min 31s


In [27]:
## rolling trend index
from collections import OrderedDict
class foctor_trend_index_period(factor_template):
    factor_name = "trend.index.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(10,13)))
    ])
    
    def formula(self, data, period):
        aa = zero_divide(abs(data["wpr"]-data["wpr"].shift(period)), data["max."+str(period)]-data["min."+str(period)])
        aa[0:period]=0
        return aa

In [28]:
x23 = foctor_trend_index_period()
for product in product_list:
    create_signal_path(x23, product, SAVE_PATH);


e:/intern/tmp pkl/btc.usd.td/trend.index.1024
e:/intern/tmp pkl/btc.usd.td/trend.index.2048
e:/intern/tmp pkl/btc.usd.td/trend.index.4096
e:/intern/tmp pkl/eth.usd.td/trend.index.1024
e:/intern/tmp pkl/eth.usd.td/trend.index.2048
e:/intern/tmp pkl/eth.usd.td/trend.index.4096
e:/intern/tmp pkl/btc.usdt/trend.index.1024
e:/intern/tmp pkl/btc.usdt/trend.index.2048
e:/intern/tmp pkl/btc.usdt/trend.index.4096
e:/intern/tmp pkl/eth.usdt/trend.index.1024
e:/intern/tmp pkl/eth.usdt/trend.index.2048
e:/intern/tmp pkl/eth.usdt/trend.index.4096


- Then we construt these range signals 

In [29]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, os.listdir(DATA_PATH + product)))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x20, product=product, HEAD_PATH=SAVE_PATH, n=14)
;
# Wall time: 25min 21s


Wall time: 25min 21s


''

In [30]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, os.listdir(DATA_PATH + product)))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x23, product=product, HEAD_PATH=SAVE_PATH,n=14)
;

## Wall time: 24min 12s

Wall time: 24min 12s


''

- Now we can combine directional signals and range signals together to become new direction signals

- For example, we have n directional signals, and m range signals, 
- then we can generate new n*m directional signals

In [34]:
## generate composite signal: trend signal * range signal = new trend signal
def construct_composite_signal(dire_signal, range_signal, period_list, product_list, HEAD_PATH):
    from collections import OrderedDict
    class foctor_xx_period(factor_template):
        factor_name = dire_signal+"."+range_signal+".period" ## name of new signal
        params = OrderedDict([
            ("period", period_list)
        ])
        def formula(self, data, period):
            return (data[dire_signal+"."+str(period)]*data[range_signal+"."+str(period)]).values ## calculation of new signal
    xx = foctor_xx_period()
    for product in product_list:
        create_signal_path(xx, product, HEAD_PATH)
        file_list = list(map(lambda x: DATA_PATH+product+"/"+x, os.listdir(DATA_PATH + product)))
        parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=xx, product=product, HEAD_PATH=HEAD_PATH,n=14)

In [9]:
dire_signal_list = ["nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"]
range_signal_list = ["range", "std", "trend.index"]
period_list = np.power(2, range(10,13))

In [39]:
product = "btc.usd.td"
file_list = list(map(lambda x: DATA_PATH+product+"/"+x, os.listdir(DATA_PATH + product)))

In [55]:
%%time
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
            construct_composite_signal(dire_signal, range_signal, period_list, product_list, SAVE_PATH)
##　Wall time:5h 25min 25s

e:/intern/tmp pkl/btc.usd.td/nr.std.1024
e:/intern/tmp pkl/btc.usd.td/nr.std.2048
e:/intern/tmp pkl/btc.usd.td/nr.std.4096
e:/intern/tmp pkl/eth.usd.td/nr.std.1024
e:/intern/tmp pkl/eth.usd.td/nr.std.2048
e:/intern/tmp pkl/eth.usd.td/nr.std.4096
e:/intern/tmp pkl/btc.usdt/nr.std.1024
e:/intern/tmp pkl/btc.usdt/nr.std.2048
e:/intern/tmp pkl/btc.usdt/nr.std.4096
e:/intern/tmp pkl/eth.usdt/nr.std.1024
e:/intern/tmp pkl/eth.usdt/nr.std.2048
e:/intern/tmp pkl/eth.usdt/nr.std.4096
e:/intern/tmp pkl/btc.usd.td/dbook.std.1024
e:/intern/tmp pkl/btc.usd.td/dbook.std.2048
e:/intern/tmp pkl/btc.usd.td/dbook.std.4096
e:/intern/tmp pkl/eth.usd.td/dbook.std.1024
e:/intern/tmp pkl/eth.usd.td/dbook.std.2048
e:/intern/tmp pkl/eth.usd.td/dbook.std.4096
e:/intern/tmp pkl/btc.usdt/dbook.std.1024
e:/intern/tmp pkl/btc.usdt/dbook.std.2048
e:/intern/tmp pkl/btc.usdt/dbook.std.4096
e:/intern/tmp pkl/eth.usdt/dbook.std.1024
e:/intern/tmp pkl/eth.usdt/dbook.std.2048
e:/intern/tmp pkl/eth.usdt/dbook.std.4096
e:/i

- Now we can combine signals of each day in to a vector
- here we use 4096 as period
- so to keep them independent, we choose 1 out of every 4096 points

In [10]:
period = 4096
os.makedirs(HEAD_PATH+"/all signal", exist_ok=True)
dire_signal_list = ["nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"]
range_signal_list = ["", "range", "std", "trend.index"]
all_signal_list = np.array([])
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
        if len(range_signal)==0:
            signal_name = dire_signal
        else:
            signal_name = dire_signal+"."+range_signal
        all_signal_list = np.append(all_signal_list,signal_name)
all_period_signal = [signal+".4096" for signal in all_signal_list]
len(all_period_signal)
## 28

28

- we have 9 directional (trend) signals and 4 range signals
- so we have 36 composite signals 
- plus the 9 original ones we have 45 signals together


In [57]:
## save the good column to separte files
def get_good(date, product, HEAD_PATH, SAVE_PATH):
    data = load(HEAD_PATH+"/pkl tick/"+product+"/"+date)
    good = data["good"]
    save(good, SAVE_PATH+"/good pkl/"+product+"/"+date)  

In [58]:
os.makedirs(SAVE_PATH+"/good pkl", exist_ok=True)
for product in product_list:
    os.makedirs(SAVE_PATH+"/good pkl/"+product, exist_ok=True)

In [59]:
%%time
for product in product_list:
    all_dates = os.listdir(DATA_PATH + product)
    parLapply(CORE_NUM, all_dates, get_good, product=product, HEAD_PATH=HEAD_PATH, SAVE_PATH="e:/intern")
    
# Wall time: 24min 22s

Wall time: 24min 22s


In [60]:
def moving_average(a, n=3) :
    ret_sum = np.cumsum(a, dtype=float)
    ret = a
    ret[n:] = (ret_sum[n:] - ret_sum[:-n])/n
    return ret

In [61]:
## parallel generate the distribution of a signal
def par_get_all_signal(signal_name, file_list, product, period, HEAD_PATH="d:/intern", SAVE_PATH="e:/intern", DATA_PATH="d:/intern"):
    n_files = len(file_list)
    all_signal = np.array([])
    for file in file_list:
        S = load(HEAD_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file) ## signal
        good = load(DATA_PATH+"/good pkl/"+product+"/"+file) ## good singal
        signal = S[good]
        moving_average(signal,period)
        chosen = (np.arange(len(signal))+1) % period==0
        all_signal = np.concatenate((all_signal, signal[chosen]), axis=0)
    save(all_signal, SAVE_PATH+"/all signal/"+product+"."+signal_name+".pkl")

In [62]:
%%time
for product in product_list:
    all_dates = os.listdir(DATA_PATH + product)
    parLapply(CORE_NUM, all_period_signal, par_get_all_signal, file_list=all_dates, product=product, period=4096, HEAD_PATH=SAVE_PATH,
             SAVE_PATH = "e:/intern", DATA_PATH="e:/intern");
## Wall time: 36min 15s


Wall time: 36min 15s


- then we can backtest our signals 
- we save the results

In [11]:
## this is to evalute the performance of a signal on a product
def evaluate_signal(signal, all_dates, product, min_pnl, min_num, 
                    CORE_NUM, HEAD_PATH="d:/intern", SIGNAL_PATH="d:/intern", period=4096, split_str="2018", tranct=1.1e-4, 
                    max_spread=0.61, tranct_ratio=True, atr_filter=0, save_path="signal result",reverse=0):
    signal_name = signal+"."+str(period) ## signal name, with period
    all_signal = load(SIGNAL_PATH+"/all signal/"+product+"."+signal_name+".pkl") ## get the distribution of the signal
    tranct = product_info[product]["tranct"] ## transaction cost of the product
    tranct_ratio = product_info[product]["tranct.ratio"] ## True: based on notional; False: fixed tranct
    open_list = np.quantile(abs(all_signal), np.append(np.arange(0.991,0.999,0.001),np.arange(0.9991,0.9999,0.0001))) ## open threshold
    thre_list = []
    for cartesian in itertools.product(open_list, np.array([0.2, 0.4, 0.6, 0.8, 1.0])): ## close threshold
        thre_list.append((cartesian[0], -cartesian[0] * cartesian[1]))
    thre_list = np.array(thre_list)
    thre_mat = pd.DataFrame(data=OrderedDict([("open", thre_list[:, 0]), ("close", thre_list[:, 1])])) ## threshold matrix
    
    if reverse>=0: ## trending signal
        print("reverse=1")
        trend_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=1, tranct=tranct, 
                                   max_spread=max_spread, tranct_ratio=tranct_ratio, min_pnl=min_pnl, min_num=min_num, atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SIGNAL_PATH=SIGNAL_PATH)
    if reverse<=0: ## reversal signal
        print("reverse=-1")
        reverse_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=-1, tranct=tranct, 
                        max_spread=max_spread, tranct_ratio=tranct_ratio, min_pnl=min_pnl, min_num=min_num, atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SIGNAL_PATH=SIGNAL_PATH)
    if reverse==0: ## both trending and reversal
        stat_result = OrderedDict([("trend.signal.stat", trend_signal_stat), ("reverse.signal.stat", reverse_signal_stat)])    
        save(stat_result, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".pkl")
    elif reverse==1: ## just trend
        save(trend_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".trend.pkl")
    elif reverse==-1: ## just reversal
        save(reverse_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".reverse.pkl")

In [12]:
## get the statistics of backtesting a signal
from collections import OrderedDict
def get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str="2018", reverse=1, tranct=1.1e-4, 
                    max_spread=0.61, tranct_ratio=True, min_pnl=2, min_num=20, atr_filter=0, HEAD_PATH="d:/intern", SIGNAL_PATH="d:/intern"):
    train_sample = all_dates<split_str ## training samples
    test_sample = all_dates>split_str ## testing samples
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                 reverse=reverse, tranct=tranct, max_spread=max_spread, tranct_ratio=tranct_ratio, 
                                  HEAD_PATH=HEAD_PATH, SIGNAL_PATH=SIGNAL_PATH, atr_filter=atr_filter)
        train_result = compute([delayed(f_par)(file) for file in all_dates[train_sample]])[0] ## get training result
    train_stat = get_hft_summary(train_result, thre_mat, sum(train_sample)) ## get training result statistics
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                  reverse=reverse, tranct=tranct, max_spread=max_spread, tranct_ratio=tranct_ratio, 
                                  HEAD_PATH=HEAD_PATH, SIGNAL_PATH=SIGNAL_PATH, atr_filter=atr_filter) 
        test_result = compute([delayed(f_par)(file) for file in all_dates[test_sample]])[0] ## get testing result
    test_stat = get_hft_summary(test_result, thre_mat, sum(test_sample)) ## get testing result statistics
    return OrderedDict([("train.stat", train_stat), ("test.stat", test_stat)])

In [147]:
product = "eth.usd.td"
signal = "nr"
split_str = "2020"
all_dates = np.array(os.listdir(DATA_PATH + product))
train_sample = all_dates<split_str ## training samples
test_sample = all_dates>split_str ## testing samples
signal_name = signal+"."+str(period) ## signal name, with period
period = 4096
all_signal = load(SIGNAL_PATH+"/all signal/"+product+"."+signal_name+".pkl") ## get the distribution of the signal
tranct = product_info[product]["tranct"] ## transaction cost of the product
spread = product_info[product]["spread"] ## transaction cost of the product
tranct_ratio = product_info[product]["tranct.ratio"] ## True: based on notional; False: fixed tranct
open_list = np.quantile(abs(all_signal), np.append(np.arange(0.991,0.999,0.001),np.arange(0.9991,0.9999,0.0001))) ## open threshold
thre_list = []
for cartesian in itertools.product(open_list, np.array([0.2, 0.4, 0.6, 0.8, 1.0])): ## close threshold
    thre_list.append((cartesian[0], -cartesian[0] * cartesian[1]))
thre_list = np.array(thre_list)
thre_mat = pd.DataFrame(data=OrderedDict([("open", thre_list[:, 0]), ("close", thre_list[:, 1])])) ## threshold matrix
reverse = 1
max_spread = spread*1.1
atr_filter = 20/spread
with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
    f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                             reverse=reverse, tranct=tranct, max_spread=max_spread, tranct_ratio=tranct_ratio, 
                              HEAD_PATH=HEAD_PATH, SIGNAL_PATH=SIGNAL_PATH, atr_filter=atr_filter)
    train_result = compute([delayed(f_par)(file) for file in all_dates[train_sample]])[0] ## get training result
train_stat = get_hft_summary(train_result, thre_mat, sum(train_sample)) ## get training result statistics


In [13]:
%%time
SIGNAL_PATH = "e:/intern"
for product in product_list[2:]:
    spread = product_info[product]["spread"]
    tranct = product_info[product]["tranct"]
    tranct_ratio = product_info[product]["tranct.ratio"]
    all_dates = np.array(os.listdir(DATA_PATH + product))
    for signal in all_signal_list:
        if product=="btc.usdt":
            if signal in ["nr", "dbook", "range.pos", "price.osci"]:
                continue
        print(product, signal)
        
        evaluate_signal(signal, all_dates, product, 2/spread, 20, CORE_NUM, HEAD_PATH, SIGNAL_PATH,
                        period=4096, split_str="2020", tranct=tranct, 
                max_spread=spread*1.1, tranct_ratio=tranct_ratio, atr_filter=20/spread, save_path="signal result atr")

## Wall time: 14h 53min 10s

btc.usdt ma.dif.10


d:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


reverse=1
reverse=-1
btc.usdt kdj.k
reverse=1
reverse=-1
btc.usdt kdj.j
reverse=1
reverse=-1
btc.usdt nr.range


d:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


reverse=1
reverse=-1
btc.usdt dbook.range
reverse=1


...

reverse=-1
eth.usdt kdj.k.trend.index
reverse=1


d:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


reverse=-1
eth.usdt kdj.j.trend.index
reverse=1


d:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


reverse=-1
Wall time: 2h 34min 9s


- we can write a function to get all the signals' performance

- we can get the result with atr and without atr separately

In [14]:
## get the signal performance
## including trend and reverse signals
def get_signal_performance_result(all_period_signal, signal_dire, period, product_list):
    trend_signal_result = pd.DataFrame(data=OrderedDict([("signal", all_period_signal), ("reverse",1),
                               ("num", 0), ("trainSharpe", 0), ("testSharpe", 0)]))
    reverse_signal_result = pd.DataFrame(data=OrderedDict([("signal", all_period_signal), ("reverse",-1),
                               ("num", 0), ("trainSharpe", 0), ("testSharpe", 0)]))
    n_signal = len(all_period_signal) ## number of all signals
    trend_train_sharpe = np.zeros(len(product_list))
    trend_test_sharpe = np.zeros(len(product_list))
    reverse_train_sharpe = np.zeros(len(product_list))
    reverse_test_sharpe = np.zeros(len(product_list))
    for k in range(n_signal):
        signal_name = all_period_signal[k]
        i = 0
        for product in product_list:
            all_dates = np.array(os.listdir(DATA_PATH + product))
            train_sample = all_dates<"2020" 
            test_sample = all_dates>"2020"
            stat_result = load(HEAD_PATH+"/" + signal_dire +"/"+product+"."+signal_name+".pkl") ## statistics of signal over a product
            trend_signal_stat = stat_result['trend.signal.stat'] ## trending statistics
            spread = product_info[product]["spread"]            
            if product=="btc.usd.td" or product=="eth.usd.td":
                avg_pnl = 10
            else:
                avg_pnl = 200

            if tuple(trend_signal_stat.keys())[0]=='train.stat':
                train_stat = trend_signal_stat["train.stat"]
                test_stat = trend_signal_stat["test.stat"]
                #good_strat = trend_signal_stat["good.strat"]
                good_strat = (train_stat["final.result"]["avg.pnl"]>avg_pnl*spread) & (train_stat["final.result"]["num"]>10) ## filter criterion
                if sum(good_strat)>2:
                    train_stat = trend_signal_stat["train.stat"]
                    test_stat = trend_signal_stat["test.stat"]
                    train_pnl = train_stat["daily.ret"].loc[:, good_strat].sum(axis=1)/sum(good_strat) ## get the daily return
                    #train_std = np.std(train_pnl)
                    #train_pnl = train_pnl/train_std
                    test_pnl = test_stat["daily.ret"].loc[:, good_strat].sum(axis=1)/sum(good_strat)
                    trend_train_sharpe[i] = sharpe(train_pnl)
                    trend_test_sharpe[i] = sharpe(test_pnl)
                    #print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))
                    i = i+1
            if i>0: ## if there are any good products
                trend_signal_result.loc[k, ("signal", "num", "trainSharpe", "testSharpe")] = (signal_name, i,  np.mean(trend_train_sharpe[:i]),np.mean(trend_test_sharpe[:i]))
        i = 0
        for product in product_list:
            stat_result = load(HEAD_PATH+"/"+signal_dire+"/"+product+"."+signal_name+".pkl")
            reverse_signal_stat = stat_result['reverse.signal.stat']
            spread = product_info[product]["spread"]
            if tuple(reverse_signal_stat.keys())[0]=='train.stat':
                #good_strat = reverse_signal_stat["good.strat"]
                train_stat = reverse_signal_stat["train.stat"]
                train_pnl = train_stat["daily.ret"].loc[:, good_strat].sum(axis=1)/sum(good_strat) ## get the daily return
                test_stat = reverse_signal_stat["test.stat"]
                good_strat = (train_stat["final.result"]["avg.pnl"]>avg_pnl*spread) & (train_stat["final.result"]["num"]>10)
                test_pnl = test_stat["daily.ret"].loc[:, good_strat].sum(axis=1)/sum(good_strat)
                if sum(good_strat)>2:
                    #train_std = np.std(train_pnl)
                    #train_pnl = train_pnl/train_std
                    reverse_train_sharpe[i] = sharpe(train_pnl)
                    reverse_test_sharpe[i] = sharpe(test_pnl)
                    i = i+1
            if i>0:
                
                reverse_signal_result.loc[k, ("signal","num", "trainSharpe", "testSharpe")] = (signal_name, i, np.mean(reverse_train_sharpe[:i]),np.mean(reverse_test_sharpe[:i]))
    return OrderedDict([("trend.signal.stat", trend_signal_result), 
                        ("reverse.signal.stat", reverse_signal_result)])


In [15]:
result_atr = get_signal_performance_result(all_period_signal, "signal result atr", 4096, product_list)

In [16]:
def get_single_signal_stat(result_atr, min_num=0, min_sharpe=0.5):
    good_trend = (result_atr["trend.signal.stat"]["num"]>min_num) & (result_atr["trend.signal.stat"]["trainSharpe"]>min_sharpe) & (result_atr["trend.signal.stat"]["testSharpe"]>min_sharpe)
    good_reverse = (result_atr["reverse.signal.stat"]["num"]>min_num) & (result_atr["reverse.signal.stat"]["trainSharpe"]>min_sharpe) & (result_atr["reverse.signal.stat"]["testSharpe"]>min_sharpe)
    print("with atr good signals: "+ str(sum(good_trend | good_reverse)))
    train_sharpe = np.mean(np.append(result_atr["trend.signal.stat"]["trainSharpe"][good_trend],
                      result_atr["reverse.signal.stat"]["trainSharpe"][good_reverse]))
    test_sharpe = np.mean(np.append(result_atr["trend.signal.stat"]["testSharpe"][good_trend],
                      result_atr["reverse.signal.stat"]["testSharpe"][good_reverse]))
    print("train sharpe: ", train_sharpe)
    print("test sharpe: ", test_sharpe)
    print(all_signal_list[good_trend])
    print(all_signal_list[good_reverse])


In [17]:
get_single_signal_stat(result_atr, min_num=0, min_sharpe=0.5)

# with atr good signals: 11
# train sharpe:  1.336956095485125
# test sharpe:  1.617636997810436
# ['dbook' 'range.pos' 'kdj.k' 'kdj.j' 'dbook.range' 'price.osci.range'
#  'ma.dif.10.range' 'nr.std' 'dbook.std' 'price.osci.std' 'ma.dif.10.std']
# []







with atr good signals: 11
train sharpe:  1.336956095485125
test sharpe:  1.617636997810436
['dbook' 'range.pos' 'kdj.k' 'kdj.j' 'dbook.range' 'price.osci.range'
 'ma.dif.10.range' 'nr.std' 'dbook.std' 'price.osci.std' 'ma.dif.10.std']
[]


In [18]:
get_single_signal_stat(result_atr, min_num=1, min_sharpe=0.5)

# with atr good signals: 3
# train sharpe:  2.191348526487736
# test sharpe:  1.765902769424989
# ['range.pos' 'kdj.k' 'kdj.j']
# []




with atr good signals: 3
train sharpe:  2.191348526487736
test sharpe:  1.765902769424989
['range.pos' 'kdj.k' 'kdj.j']
[]


In [19]:
get_single_signal_stat(result_atr, min_num=2, min_sharpe=0.5)

# with atr good signals: 0
# train sharpe:  nan
# test sharpe:  nan
# []
# []



with atr good signals: 0
train sharpe:  nan
test sharpe:  nan
[]
[]


d:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
d:\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


- we can see that their results are very similar

- Conclusion
- This week we have generated results for all of the 35 signals
- We have compared backtest result with and without atr
- we find that there is not much difference between them
- previously we found that for iorn with atr is better than without atr
- that's because we lower the threhsold for number of trades
- now we use 20 trades for training maybe for i there are not enough trades
- Anyway, currently there are too many signals and hard to tune parameters
- in the future, we can use fewer signals so that we can do the research more efficiently